In [ ]:
'''
БАГИ:
- ИЗ-ЗА НЕПОЛНОТЫ ДАННЫХ ПО 1 СОЗЫВУ: в таблице результатов нет инфы об избрании 5-курсников
- научиться тянуть время коммента (реплая)
'''

In [2]:
import pandas as pd

In [537]:
def split_year_stream(year_stream_no):
    if 'm' in year_stream_no:
        year_no = int(year_stream_no[-1])
        stream_no = 0
    elif '.' in year_stream_no:
        year_no = int(year_stream_no[0])
        stream_no = int(year_stream_no[-1])
    else:
        year_no, stream_no = -1, -1
    return year_no, stream_no

# split_year_stream('m2')


In [231]:
# https://docs.python.org/3/library/datetime.html

from datetime import datetime, date, time

# поле дата-время по полям даты и времени
def dttm_by_date_time(link_date, link_time):
    link_dt = datetime.fromisoformat(link_date)
    if link_time in [None, '']:
        link_time_list = [23,59]
    else:
        link_time_list = list(map(int, link_time.split(':')))
    return datetime.combine(link_dt, time(link_time_list[0], link_time_list[1]))

# link_date = date(2012, 3, 7)
link_date = '2012-03-07'
link_time = '20:55'
link_dttm = dttm_by_date_time(link_date, link_time)
link_dttm


datetime.datetime(2012, 3, 7, 20, 55)

# 1. To fix CANDIDATE tables

In [196]:

ssmm_announcement_conv_1_3 = pd.read_csv('ssmm_announcement_info_conv_1_3.csv', sep='\t', encoding='utf-8')
candidates_conv_1_3 = ssmm_announcement_conv_1_3[['name','convocation_no','year_stream_no','year_no','stream_no','academic_group_no','program_txt','link_date','link_time','link']]

candidates_conv_1_3

,name,convocation_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link_date,link_time,link
0,Левин Андрей,1,1.1,1,1,101.0,Активный участник и организатор различных меро...,2012-02-26,0:16,https://vk.com/topic-33327655_26089953
1,Матвеев Дмитрий Александрович,1,1.1,1,1,101.0,NaN,2012-02-26,0:16,https://vk.com/topic-33327655_26089953
2,Власов Артем Андреевич,1,1.2,1,2,107.0,NaN,2012-02-26,0:16,https://vk.com/topic-33327655_26089953
3,Кириллова Полина,1,1.2,1,2,112.0,Мне нравится заниматься общественной деятельно...,2012-02-26,0:16,https://vk.com/topic-33327655_26089953
4,Семенов Павел,1,1.2,1,2,112.0,"""Здравствуйте. Я так понимаю, что я уже поздно...",2012-02-26,0:16,https://vk.com/topic-33327655_26089953
...,...,...,...,...,...,...,...,...,...,...
126,Сапунов Кирилл Вячеславович,3,5.3,5,3,525.0,NaN,2013-11-17,14:21,https://vk.com/topic-33327655_29117817
127,Самонов Алексей Александрович,3,5.3,5,3,526.0,"""Я студент 5 курса, есть опыт в организации ра...",2013-11-17,14:21,https://vk.com/topic-33327655_29117817
128,Короткин Виталий Владимирович,3,m1,1,0,NaN,"""с сентября 2012-н.в. являюсь председателем СК...",2013-11-17,14:16,https://vk.com/topic-33327655_29117788
129,Буряк Анна Андреевна,3,m1,1,0,NaN,"""Состояла в студсовете в 2010-2011 годах, на п...",2013-11-17,14:16,https://vk.com/topic-33327655_29117788


In [191]:
ssmm_announcements_conv_4_11 = pd.read_csv('ssmm_announcement_info_conv_4_11.csv', sep='\t', encoding='utf-8')
# ssmm_announcements_conv_4_11.head()

In [194]:
def conv_no_by_year(year):
    if year >= 2013:
        return year-2010
    else:
        return -1

candidates_conv_4_11 = pd.read_csv('ssmm_candidates_info_conv_4_11.csv', sep='\t', encoding='utf-8')
candidates_conv_4_11['convocation_no'] = candidates_conv_4_11['link_year_no'].apply(lambda x: conv_no_by_year(x))
candidates_conv_4_11 = candidates_conv_4_11.merge(ssmm_announcements_conv_4_11[['link','link_date','link_time']], left_on='link',right_on='link',how='left')
candidates_conv_4_11 = candidates_conv_4_11[['name','convocation_no','year_stream_no','year_no','stream_no','academic_group_no','program_txt','link_date','link_time','link']]

candidates_conv_4_11


,name,convocation_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link_date,link_time,link
0,Суркова Анастасия Юрьевна,4,2.2,2,2,209.0,"«Подавая заявку в прошлом году в студсовет, яв...",2014-11-13,21:39,https://vk.com/wall-33327655_2202
1,Денисов Илья Олегович,4,2.2,2,2,207.0,"«Вечер добрый, коллеги!\nЯ бы хотел подать зая...",2014-11-13,21:39,https://vk.com/wall-33327655_2202
2,Тюрина Татьяна Игоревна,4,2.2,2,2,209.0,«Уже во второй раз избираюсь в студенческий со...,2014-11-13,21:39,https://vk.com/wall-33327655_2202
3,Дорман Елена Вячеславовна,4,2.3,2,3,226.0,"«По моему мнению, за прошедший год я не плохо ...",2014-11-13,21:40,https://vk.com/wall-33327655_2203
4,Оськин Александр Андреевич,4,2.3,2,3,225.0,«Прежде всего здравствуйте:)\nНа протяжении по...,2014-11-13,21:40,https://vk.com/wall-33327655_2203
...,...,...,...,...,...,...,...,...,...,...
385,Терехов Михаил Сергеевич,11,6.1,6,1,602.0,За правду!,2021-10-16,10:25,https://vk.com/wall-33327655_9978
386,Поликарпов Александр Валерьевич,11,6.3,6,3,621.0,Был одним из основателей бегового клуба MSU RU...,2021-10-16,10:50,https://vk.com/wall-33327655_9982
387,Каляев Тимур Джанбулатович,11,m1,1,0,NaN,"Всем привет! Я только недавно на мехмате, но х...",2021-10-16,11:20,https://vk.com/wall-33327655_9983
388,Маммагаджиев Руслан Гаджимагомедович,11,m1,1,0,NaN,Долгое время был пикчером профкома мехмата. Се...,2021-10-16,11:20,https://vk.com/wall-33327655_9983


In [ ]:
# ssmm_announcements_conv_1_3 = pd.read_csv('ssmm_announcement_info_conv_1_3.csv', sep='\t', encoding='utf-8')
# ssmm_announcements_conv_1_3.head()

In [197]:
list_of_df = [candidates_conv_1_3,candidates_conv_4_11]
candidates_conv_1_11 = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
candidates_conv_1_11


,name,convocation_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link_date,link_time,link
0,Левин Андрей,1,1.1,1,1,101.0,Активный участник и организатор различных меро...,2012-02-26,0:16,https://vk.com/topic-33327655_26089953
1,Матвеев Дмитрий Александрович,1,1.1,1,1,101.0,NaN,2012-02-26,0:16,https://vk.com/topic-33327655_26089953
2,Власов Артем Андреевич,1,1.2,1,2,107.0,NaN,2012-02-26,0:16,https://vk.com/topic-33327655_26089953
3,Кириллова Полина,1,1.2,1,2,112.0,Мне нравится заниматься общественной деятельно...,2012-02-26,0:16,https://vk.com/topic-33327655_26089953
4,Семенов Павел,1,1.2,1,2,112.0,"""Здравствуйте. Я так понимаю, что я уже поздно...",2012-02-26,0:16,https://vk.com/topic-33327655_26089953
...,...,...,...,...,...,...,...,...,...,...
516,Терехов Михаил Сергеевич,11,6.1,6,1,602.0,За правду!,2021-10-16,10:25,https://vk.com/wall-33327655_9978
517,Поликарпов Александр Валерьевич,11,6.3,6,3,621.0,Был одним из основателей бегового клуба MSU RU...,2021-10-16,10:50,https://vk.com/wall-33327655_9982
518,Каляев Тимур Джанбулатович,11,m1,1,0,NaN,"Всем привет! Я только недавно на мехмате, но х...",2021-10-16,11:20,https://vk.com/wall-33327655_9983
519,Маммагаджиев Руслан Гаджимагомедович,11,m1,1,0,NaN,Долгое время был пикчером профкома мехмата. Се...,2021-10-16,11:20,https://vk.com/wall-33327655_9983


In [198]:

def key_by_2_fields(f1, f2):
    return str(f1)+'_'+'_'.join(str(f2).split())


In [374]:
candidates_conv_1_11['conv_x_name'] = candidates_conv_1_11.apply(lambda x: key_by_2_fields(x['convocation_no'],x['name']), axis=1)

candidates_conv_1_11 = candidates_conv_1_11.fillna(value={'link_time': '23:59'})
candidates_conv_1_11['link_dttm'] = candidates_conv_1_11.apply(lambda x: dttm_by_date_time(x['link_date'],str(x['link_time'])), axis=1)
candidates_conv_1_11 = candidates_conv_1_11.drop_duplicates()
candidates_conv_1_11


,name,convocation_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link_date,link_time,link,conv_x_name,link_dttm
0,Левин Андрей,1,1.1,1,1,101.0,Активный участник и организатор различных меро...,2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Левин_Андрей,2012-02-26 00:16:00
1,Матвеев Дмитрий Александрович,1,1.1,1,1,101.0,NaN,2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Матвеев_Дмитрий_Александрович,2012-02-26 00:16:00
2,Власов Артем Андреевич,1,1.2,1,2,107.0,NaN,2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Власов_Артем_Андреевич,2012-02-26 00:16:00
3,Кириллова Полина,1,1.2,1,2,112.0,Мне нравится заниматься общественной деятельно...,2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Кириллова_Полина,2012-02-26 00:16:00
4,Семенов Павел,1,1.2,1,2,112.0,"""Здравствуйте. Я так понимаю, что я уже поздно...",2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Семенов_Павел,2012-02-26 00:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...
516,Терехов Михаил Сергеевич,11,6.1,6,1,602.0,За правду!,2021-10-16,10:25,https://vk.com/wall-33327655_9978,11_Терехов_Михаил_Сергеевич,2021-10-16 10:25:00
517,Поликарпов Александр Валерьевич,11,6.3,6,3,621.0,Был одним из основателей бегового клуба MSU RU...,2021-10-16,10:50,https://vk.com/wall-33327655_9982,11_Поликарпов_Александр_Валерьевич,2021-10-16 10:50:00
518,Каляев Тимур Джанбулатович,11,m1,1,0,NaN,"Всем привет! Я только недавно на мехмате, но х...",2021-10-16,11:20,https://vk.com/wall-33327655_9983,11_Каляев_Тимур_Джанбулатович,2021-10-16 11:20:00
519,Маммагаджиев Руслан Гаджимагомедович,11,m1,1,0,NaN,Долгое время был пикчером профкома мехмата. Се...,2021-10-16,11:20,https://vk.com/wall-33327655_9983,11_Маммагаджиев_Руслан_Гаджимагомедович,2021-10-16 11:20:00


In [234]:
candidates_conv_1_11.to_csv('ssmm_candidates_info_conv_1_11.csv', sep='\t', encoding='utf-8', index=False)

# ПЕРВАЯ ГЛАВНАЯ ТАБЛИЦА

# 2. To fix VOTES tables

In [380]:

ssmm_votes_conv_1 = pd.read_csv('ssmm_votes_info_conv_1.csv', sep='\t', encoding='utf-8')
ssmm_votes_conv_2 = pd.read_csv('ssmm_votes_info_conv_2.csv', sep='\t', encoding='utf-8')
ssmm_votes_conv_3 = pd.read_csv('ssmm_votes_info_conv_3.csv', sep='\t', encoding='utf-8')
ssmm_votes_conv_4_11 = pd.read_csv('ssmm_votes_info_conv_4_11.csv', sep='\t', encoding='utf-8')


In [169]:
ssmm_votes_conv_4_11.head()

,link_year_no_x_name,name,link_year_no,year_stream_no,year_no,stream_no,votes_cnt,elected_flg,link,convocation_no
0,2014_Суркова_Анастасия_Юрьевна,Суркова Анастасия Юрьевна,2014,2.2,2,2,53,1,https://vk.com/wall-33327655_2215,4
1,2014_Денисов_Илья_Олегович,Денисов Илья Олегович,2014,2.2,2,2,61,1,https://vk.com/wall-33327655_2215,4
2,2014_Тюрина_Татьяна_Игоревна,Тюрина Татьяна Игоревна,2014,2.2,2,2,48,1,https://vk.com/wall-33327655_2215,4
3,2014_Дорман_Елена_Вячеславовна,Дорман Елена Вячеславовна,2014,2.3,2,3,71,1,https://vk.com/wall-33327655_2216,4
4,2014_Оськин_Александр_Андреевич,Оськин Александр Андреевич,2014,2.3,2,3,45,1,https://vk.com/wall-33327655_2216,4


In [171]:
ssmm_results_conv_4_11 = pd.read_csv('ssmm_results_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_results_conv_4_11.head()

,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no,stream_size,voted_cnt,invalid_cnt,NOTA_cnt,organizers
0,https://vk.com/wall-33327655_2250,2014_1.1,2014-11-21,2014,11,21,22:15,1.1,1,1,166,129,10,5,"['Ильёй Денисовым', 'Анастасией Сурковой']"
1,https://vk.com/wall-33327655_2252,2014_1.2,2014-11-21,2014,11,21,22:25,1.2,1,2,136,95,4,6,"['Ираклием Глунчадзе', 'Анастасией Сурковой']"
2,https://vk.com/wall-33327655_2254,2014_1.3,2014-11-21,2014,11,21,22:35,1.3,1,3,138,83,1,2,"['Ираклием Глунчадзе', 'Анастасией Савченко']"
3,https://vk.com/wall-33327655_2329,2014_2.1,2014-11-27,2014,11,27,23:15,2.1,2,1,135,89,1,1,"['Ираклием Глунчадзе', 'Анастасией Сурковой']"
4,https://vk.com/wall-33327655_2215,2014_2.2,2014-11-17,2014,11,17,21:06,2.2,2,2,135,87,0,7,['Глунчадзе Ираклием']


In [157]:
ssmm_votes_conv_1.head()

temp_df = ssmm_votes_conv_1

temp_df = temp_df[['convocation_no', 'name', 'year_stream_no', 'votes_cnt', 'elected_flg','link_date','link_time','link']] # , 'link'
temp_votes_conv_1 = temp_df

temp_votes_conv_1.head()

,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link
0,1,Матвеев Дмитрий Александрович,1.1,49,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
1,1,Левин Андрей,1.1,48,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
2,1,Кириллова Полина,1.2,40,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
3,1,Власов Артем Андреевич,1.2,36,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
4,1,Кузнецова Алёна,1.2,28,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953


In [160]:

def full_name_by_short_name1(short_name):
#     ЗАВИСИТ ОТ ВНЕШНЕЙ ТАБЛИЦЫ candidates_conv_1_11
    names_conv3 = list(candidates_conv_1_11['name'][candidates_conv_1_11['convocation_no']==1])
    for full_name in names_conv3:
        if short_name in full_name:
            return full_name

temp_df = temp_votes_conv_1
temp_df['full_name'] = temp_df['name'].apply(lambda x: full_name_by_short_name1(x))
temp_df['max_name'] = temp_df['full_name'].combine_first(temp_df['name'])
temp_df = temp_df.drop(['full_name','name'],axis=1).rename(columns={'max_name':'name'})
# temp_df[:20]

temp_votes_conv_1_upd = temp_df[['convocation_no', 'name', 'year_stream_no', 'votes_cnt', 'elected_flg','link_date','link_time','link']] # , 'link'
temp_votes_conv_1_upd

/var/folders/zh/mnvg84rj3_7413ycjzpn5y8r0000gp/T/ipykernel_44908/3982715174.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['full_name'] = temp_df['name'].apply(lambda x: full_name_by_short_name1(x))
/var/folders/zh/mnvg84rj3_7413ycjzpn5y8r0000gp/T/ipykernel_44908/3982715174.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['max_name'] = temp_df['full_name'].combine_first(temp_df['name'])


,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link
0,1,Матвеев Дмитрий Александрович,1.1,49,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
1,1,Левин Андрей,1.1,48,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
2,1,Кириллова Полина,1.2,40,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
3,1,Власов Артем Андреевич,1.2,36,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
4,1,Кузнецова Алёна,1.2,28,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
5,1,Семенов Павел,1.2,20,0.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
6,1,Тишин Павел Владимирович,1.3,51,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
7,1,Сафина Диана Ураловна,1.3,44,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
8,1,Медведева Яна Григорьевна,1.3,23,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
9,1,Кунденко Вадим,1.3,21,0.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953


In [151]:
ssmm_votes_conv_2.head()

temp_df = ssmm_votes_conv_2

temp_df['votes_cnt'] = temp_df.apply(lambda x: None)
temp_df = temp_df[['convocation_no', 'name', 'year_stream_no', 'votes_cnt', 'elected_flg', 'link_date', 'link_time', 'link']]
temp_votes_conv_2 = temp_df

temp_votes_conv_2.head()

,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link
0,2,Остроухова Наталья Владимировна,1.1,NaN,1,2012-11-19,16:13,https://vk.com/wall-33327655_725
1,2,Габидов Александр Сергеевич,1.1,NaN,1,2012-11-19,16:13,https://vk.com/wall-33327655_725
2,2,Дяченко Мария Игоревна,1.1,NaN,1,2012-11-19,16:13,https://vk.com/wall-33327655_725
3,2,Ватутин Кирилл Александрович,1.3,NaN,1,2012-11-19,16:13,https://vk.com/wall-33327655_725
4,2,Никифоров Сергей Игоревич,1.3,NaN,1,2012-11-19,16:13,https://vk.com/wall-33327655_725


In [381]:
# ssmm_votes_conv_3.head()

temp_df = ssmm_votes_conv_3

temp_df['votes_cnt'] = temp_df.apply(lambda x: None)
temp_df = temp_df[['convocation_no', 'name', 'year_stream_no', 'votes_cnt', 'elected_flg', 'link_date', 'link_time', 'link']]
temp_votes_conv_3 = temp_df

temp_votes_conv_3.head()

,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link
0,3,Алекберов,1.1,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600
1,3,Бурнус,1.1,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600
2,3,Денисов,1.2,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600
3,3,Суркова,1.2,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600
4,3,Тюрина,1.2,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600


In [382]:

def full_name_by_short_name3(short_name):
#     ЗАВИСИТ ОТ ВНЕШНЕЙ ТАБЛИЦЫ candidates_conv_1_11
    names_conv3 = list(candidates_conv_1_11['name'][candidates_conv_1_11['convocation_no']==3])
    for full_name in names_conv3:
        if short_name in full_name:
            return full_name

temp_votes_conv_3_upd = temp_votes_conv_3
temp_votes_conv_3_upd['full_name'] = temp_votes_conv_3_upd['name'].apply(lambda x: full_name_by_short_name3(x))
temp_votes_conv_3_upd['max_name'] = temp_votes_conv_3_upd['full_name'].combine_first(temp_votes_conv_3_upd['name'])
temp_votes_conv_3_upd = temp_votes_conv_3_upd[['convocation_no', 'max_name', 'year_stream_no', 'votes_cnt', 'elected_flg', 'link_date', 'link_time', 'link']].rename(columns={'max_name':'name'})
temp_votes_conv_3_upd.head()


/var/folders/zh/mnvg84rj3_7413ycjzpn5y8r0000gp/T/ipykernel_44908/4129198660.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_votes_conv_3_upd['full_name'] = temp_votes_conv_3_upd['name'].apply(lambda x: full_name_by_short_name3(x))


,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link
0,3,Алекберов,1.1,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600
1,3,Бурнус Артём Владимирович,1.1,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600
2,3,Денисов Илья Олегович,1.2,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600
3,3,Суркова Анастасия Юрьевна,1.2,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600
4,3,Тюрина Татьяна Игоревна,1.2,NaN,1,2013-12-10,1:46,https://vk.com/wall-33327655_1600


In [383]:
list_of_df = [temp_votes_conv_1_upd,temp_votes_conv_2,temp_votes_conv_3_upd]
temp_votes_conv_1_3 = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
temp_votes_conv_1_3 = temp_votes_conv_1_3[['convocation_no', 'name', 'year_stream_no', 'votes_cnt', 'elected_flg', 'link_date', 'link_time', 'link']]

final_votes_conv_1_3 = temp_votes_conv_1_3
final_votes_conv_1_3.head()

,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link
0,1,Матвеев Дмитрий Александрович,1.1,49,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
1,1,Левин Андрей,1.1,48,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
2,1,Кириллова Полина,1.2,40,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
3,1,Власов Артем Андреевич,1.2,36,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953
4,1,Кузнецова Алёна,1.2,28,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953


In [299]:
# reorder

temp_df = ssmm_votes_conv_4_11.merge(ssmm_results_conv_4_11[['link','link_date','link_time']], left_on='link', right_on='link',how='outer')
# temp_df

temp_df['convocation_no'] = temp_df['link_year_no'].apply(lambda x: conv_no_by_year(x))
temp_df = temp_df[['convocation_no', 'name', 'year_stream_no', 'votes_cnt', 'elected_flg', 'link_date', 'link_time', 'link']]
final_votes_conv_4_11 = temp_df

final_votes_conv_4_11.head()


,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link
0,4,Суркова Анастасия Юрьевна,2.2,53,1,2014-11-17,21:06,https://vk.com/wall-33327655_2215
1,4,Денисов Илья Олегович,2.2,61,1,2014-11-17,21:06,https://vk.com/wall-33327655_2215
2,4,Тюрина Татьяна Игоревна,2.2,48,1,2014-11-17,21:06,https://vk.com/wall-33327655_2215
3,4,Дорман Елена Вячеславовна,2.3,71,1,2014-11-17,21:21,https://vk.com/wall-33327655_2216
4,4,Оськин Александр Андреевич,2.3,45,1,2014-11-17,21:21,https://vk.com/wall-33327655_2216


In [384]:
list_of_df = [final_votes_conv_1_3,final_votes_conv_4_11]
final_votes_conv_1_11 = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)

final_votes_conv_1_11['conv_x_name'] = final_votes_conv_1_11.apply(lambda x: key_by_2_fields(x['convocation_no'],x['name']), axis=1)

final_votes_conv_1_11 = final_votes_conv_1_11.fillna(value={'link_time': '23:59'})
final_votes_conv_1_11['link_dttm'] = final_votes_conv_1_11.apply(lambda x: dttm_by_date_time(x['link_date'],str(x['link_time'])), axis=1)
final_votes_conv_1_11 = final_votes_conv_1_11.drop_duplicates()

final_votes_conv_1_11


,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link,conv_x_name,link_dttm
0,1,Матвеев Дмитрий Александрович,1.1,49,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Матвеев_Дмитрий_Александрович,2012-03-07 20:55:00
1,1,Левин Андрей,1.1,48,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Левин_Андрей,2012-03-07 20:55:00
2,1,Кириллова Полина,1.2,40,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Кириллова_Полина,2012-03-07 20:55:00
3,1,Власов Артем Андреевич,1.2,36,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Власов_Артем_Андреевич,2012-03-07 20:55:00
4,1,Кузнецова Алёна,1.2,28,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Кузнецова_Алёна,2012-03-07 20:55:00
...,...,...,...,...,...,...,...,...,...,...
501,11,Маммагаджиев Руслан Гаджимагомедович,m1,10,1.0,2021-10-20,20:21,https://vk.com/wall-33327655_10018,11_Маммагаджиев_Руслан_Гаджимагомедович,2021-10-20 20:21:00
502,11,Третьякова Екатерина Алексеевна,m1,19,1.0,2021-10-20,20:21,https://vk.com/wall-33327655_10018,11_Третьякова_Екатерина_Алексеевна,2021-10-20 20:21:00
503,11,Мазитова Вероника Рустэмовна,6.1,27,1.0,2021-10-21,15:44,https://vk.com/wall-33327655_10026,11_Мазитова_Вероника_Рустэмовна,2021-10-21 15:44:00
504,11,Терехов Михаил Сергеевич,6.1,16,1.0,2021-10-21,15:44,https://vk.com/wall-33327655_10026,11_Терехов_Михаил_Сергеевич,2021-10-21 15:44:00


In [385]:
final_votes_conv_1_11.to_csv('ssmm_votes_info_conv_1_11.csv', sep='\t', encoding='utf-8', index=False)

# ВТОРАЯ ГЛАВНАЯ ТАБЛИЦА

# ОСНОВНЫЕ ТАБЛИЦЫ ПО КАНДИДАТАМ

In [265]:
ssmm_candidates_info_conv_1_11 = pd.read_csv('ssmm_candidates_info_conv_1_11.csv', sep='\t', encoding='utf-8')
ssmm_candidates_info_conv_1_11.head()

,name,convocation_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link_date,link_time,link,conv_x_name,link_dttm
0,Левин Андрей,1,1.1,1,1,101.0,Активный участник и организатор различных меро...,2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Левин_Андрей,2012-02-26 00:16:00
1,Матвеев Дмитрий Александрович,1,1.1,1,1,101.0,NaN,2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Матвеев_Дмитрий_Александрович,2012-02-26 00:16:00
2,Власов Артем Андреевич,1,1.2,1,2,107.0,NaN,2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Власов_Артем_Андреевич,2012-02-26 00:16:00
3,Кириллова Полина,1,1.2,1,2,112.0,Мне нравится заниматься общественной деятельно...,2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Кириллова_Полина,2012-02-26 00:16:00
4,Семенов Павел,1,1.2,1,2,112.0,"""Здравствуйте. Я так понимаю, что я уже поздно...",2012-02-26,0:16,https://vk.com/topic-33327655_26089953,1_Семенов_Павел,2012-02-26 00:16:00


In [386]:
ssmm_votes_info_conv_1_11 = pd.read_csv('ssmm_votes_info_conv_1_11.csv', sep='\t', encoding='utf-8')
ssmm_votes_info_conv_1_11.head()

,convocation_no,name,year_stream_no,votes_cnt,elected_flg,link_date,link_time,link,conv_x_name,link_dttm
0,1,Матвеев Дмитрий Александрович,1.1,49.0,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Матвеев_Дмитрий_Александрович,2012-03-07 20:55:00
1,1,Левин Андрей,1.1,48.0,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Левин_Андрей,2012-03-07 20:55:00
2,1,Кириллова Полина,1.2,40.0,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Кириллова_Полина,2012-03-07 20:55:00
3,1,Власов Артем Андреевич,1.2,36.0,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Власов_Артем_Андреевич,2012-03-07 20:55:00
4,1,Кузнецова Алёна,1.2,28.0,1.0,2012-03-07,20:55,https://vk.com/topic-33327655_26089953,1_Кузнецова_Алёна,2012-03-07 20:55:00


In [387]:
df1 = ssmm_candidates_info_conv_1_11[['conv_x_name','name']]
df2 = ssmm_votes_info_conv_1_11[['conv_x_name','name']]

df1_x_d2 = df1.merge(df2, left_on='conv_x_name', right_on='conv_x_name',how='outer')
df1_x_d2_nan = df1_x_d2[df1_x_d2.isna().any(axis=1)]
df1_x_d2_nan


,conv_x_name,name_x,name_y
39,1_Хисматуллина_Марина_Рашитовна,Хисматуллина Марина Рашитовна,NaN
40,1_Штейников_Юрий_Николаевич,Штейников Юрий Николаевич,NaN
41,1_Савушкин_Никита_Максимович,Савушкин Никита Максимович,NaN
42,1_Миценко_Вадим_Валериевич,Миценко Вадим Валериевич,NaN
43,1_Буряк_Анна_Андреевна,Буряк Анна Андреевна,NaN
44,1_Дубровин_Григорий_Анатольевич,Дубровин Григорий Анатольевич,NaN
46,2_Николаева_Наталия_Михайловна,Николаева Наталия Михайловна,NaN
49,2_Ванунц_Арсений_Ашотович,Ванунц Арсений Ашотович,NaN
56,2_Медведева_Яна_Григорьевна,Медведева Яна Григорьевна,NaN
60,2_Сафина_Диана_Ураловна,Сафина Диана Ураловна,NaN


In [388]:
# ОБЩАЯ ТАБЛИЦА КАНДИДАТОВ С АНОНСАМИ И ГОЛОСАМИ

temp_df = ssmm_candidates_info_conv_1_11[['conv_x_name','name','convocation_no','year_stream_no','academic_group_no','program_txt','link','link_dttm']].merge(ssmm_votes_info_conv_1_11[['conv_x_name','name','convocation_no','year_stream_no','votes_cnt','elected_flg','link','link_dttm']], left_on='conv_x_name',right_on='conv_x_name',how='outer')

for word in ['name','convocation_no','year_stream_no']:
    temp_df[word] = temp_df[word+'_x'].combine_first(temp_df[word+'_y'])
    temp_df = temp_df.drop([word+'_x', word+'_y'],axis=1)

# temp_df['academic_group_no'] = temp_df['academic_group_no'].apply(lambda x: int(x))
temp_df['convocation_no'] = temp_df['convocation_no'].apply(lambda x: int(x))

temp_df = temp_df.rename(columns={'link_x':'announcement_link','link_y':'result_link','link_dttm_x':'announcement_dttm','link_dttm_y':'result_dttm'})
temp_df['appearance_dttm'] = temp_df['announcement_dttm'].combine_first(temp_df['result_dttm'])

ssmm_candidates_x_votes_info_conv_1_11 = temp_df[['conv_x_name','name','convocation_no','year_stream_no','academic_group_no','program_txt','announcement_link','announcement_dttm','votes_cnt','elected_flg','result_link','result_dttm','appearance_dttm']].drop_duplicates()
ssmm_candidates_x_votes_info_conv_1_11



,conv_x_name,name,convocation_no,year_stream_no,academic_group_no,program_txt,announcement_link,announcement_dttm,votes_cnt,elected_flg,result_link,result_dttm,appearance_dttm
0,1_Левин_Андрей,Левин Андрей,1,1.1,101.0,Активный участник и организатор различных меро...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,48.0,1.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
1,1_Матвеев_Дмитрий_Александрович,Матвеев Дмитрий Александрович,1,1.1,101.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,49.0,1.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
2,1_Власов_Артем_Андреевич,Власов Артем Андреевич,1,1.2,107.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,36.0,1.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
3,1_Кириллова_Полина,Кириллова Полина,1,1.2,112.0,Мне нравится заниматься общественной деятельно...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,40.0,1.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
4,1_Семенов_Павел,Семенов Павел,1,1.2,112.0,"""Здравствуйте. Я так понимаю, что я уже поздно...",https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,20.0,0.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,11_Маммагаджиев_Руслан_Гаджимагомедович,Маммагаджиев Руслан Гаджимагомедович,11,m1,NaN,Долгое время был пикчером профкома мехмата. Се...,https://vk.com/wall-33327655_9983,2021-10-16 11:20:00,10.0,1.0,https://vk.com/wall-33327655_10018,2021-10-20 20:21:00,2021-10-16 11:20:00
520,11_Третьякова_Екатерина_Алексеевна,Третьякова Екатерина Алексеевна,11,m1,NaN,Я как человек с немалым опытом в делах организ...,https://vk.com/wall-33327655_9983,2021-10-16 11:20:00,19.0,1.0,https://vk.com/wall-33327655_10018,2021-10-20 20:21:00,2021-10-16 11:20:00
521,1_Федорова_Елена_Игоревна,Федорова Елена Игоревна,1,4.4,NaN,NaN,NaN,NaN,20.0,1.0,https://vk.com/topic-33327655_26089874,2012-03-07 18:55:00,2012-03-07 18:55:00
522,3_Алекберов,Алекберов,3,1.1,NaN,NaN,NaN,NaN,NaN,1.0,https://vk.com/wall-33327655_1600,2013-12-10 01:46:00,2013-12-10 01:46:00


In [389]:
ssmm_candidates_x_votes_info_conv_1_11.to_csv('ssmm_candidates_x_votes_info_conv_1_11.csv', sep='\t', encoding='utf-8', index=False)

# СОВСЕМ ГЛАВНАЯ ТАБЛИЦА

In [390]:
# СТЕРЖНЕВАЯ ТАБЛИЦА КАНДИДАТОВ С ПОРЯДКОВЫМ ssmm_id

# упорядочивание по времени первого появления, при равенстве - по ФИО
full_min_dttm = ssmm_candidates_x_votes_info_conv_1_11.groupby(['name']).agg({"appearance_dttm": "min"}).reset_index().sort_values(['appearance_dttm', 'name'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)

import numpy as np

full_min_dttm['index'] = np.arange(full_min_dttm.shape[0])
full_min_dttm['ssmm_id'] = full_min_dttm['index'].apply(lambda x: x+1)
ssmm_candidates_core = full_min_dttm.drop(['index'], axis=1)
ssmm_candidates_core = ssmm_candidates_core[['ssmm_id', 'name', 'appearance_dttm']].rename(columns={'name':'candidate_name','appearance_dttm':'first_appearance_dttm'})

ssmm_candidates_core
# ssmm_candidates_core[:60]


,ssmm_id,candidate_name,first_appearance_dttm
0,1,Владыкина Вероника Евгеньевна,2012-02-25 23:16:00
1,2,Коновалов Антон Александрович,2012-02-25 23:16:00
2,3,Ласкин Алексей Александрович,2012-02-25 23:16:00
3,4,Малхасян Арамаис,2012-02-25 23:16:00
4,5,Попов Леонид Андреевич,2012-02-25 23:16:00
...,...,...,...
315,316,Уваров Андрей Михайлович,2021-10-15 11:11:00
316,317,Поликарпов Александр Валерьевич,2021-10-16 10:50:00
317,318,Каляев Тимур Джанбулатович,2021-10-16 11:20:00
318,319,Маммагаджиев Руслан Гаджимагомедович,2021-10-16 11:20:00


In [391]:
ssmm_candidates_core.to_csv('ssmm_candidates_core.csv', sep='\t', encoding='utf-8', index=False)


In [392]:
# member_id - номер кандидата в порядке избрания в члены

temd_df = ssmm_candidates_x_votes_info_conv_1_11
elected_min_dttm = temd_df[temd_df['elected_flg'] == 1].groupby(['name']).agg({"result_dttm": "min"}).reset_index().sort_values(['result_dttm', 'name'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)

import numpy as np

elected_min_dttm['index'] = np.arange(elected_min_dttm.shape[0])
elected_min_dttm['member_id'] = elected_min_dttm['index'].apply(lambda x: x+1)
ssmm_members_core = elected_min_dttm.drop(['index'], axis=1)
ssmm_members_core = ssmm_members_core[['member_id', 'name', 'result_dttm']].rename(columns={'name':'member_name','result_dttm':'first_result_dttm'})

# ssmm_members_core
ssmm_members_core[:30]


,member_id,member_name,first_result_dttm
0,1,Аканаев Тимур Рашитович,2012-02-28 22:34:00
1,2,Асташов Дмитрий Сергеевич,2012-02-28 22:34:00
2,3,Гильманов Роман Павлович,2012-02-28 22:34:00
3,4,Глухова Алёна Олеговна,2012-02-28 22:34:00
4,5,Костицын Александр Алексеевич,2012-02-28 22:34:00
5,6,Матвеев Алексей Викторович,2012-02-28 22:34:00
6,7,Осин Руслан Владимирович,2012-02-28 22:34:00
7,8,Харлашин Владимир Сергеевич,2012-02-28 22:34:00
8,9,Шариков Алексей Анатольевич,2012-02-28 22:34:00
9,10,Владыкина Вероника Евгеньевна,2012-03-07 02:03:00


## ФЛАГИ КАНДИДАТОВ

In [536]:
# флаги о членстве
# - сколько раз избирался
# - первый созыв
# - последний созыв
# - список созывов

temd_df = ssmm_candidates_x_votes_info_conv_1_11
ssmm_membres_x_conv = temd_df[temd_df['elected_flg'] == 1].sort_values(['convocation_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)

ssmm_membres_conv = ssmm_membres_x_conv.groupby('name').agg({'convocation_no':['min','max', lambda x: list(x), lambda x: len(list(x))]}) 

ssmm_membres_conv.columns = ['_'.join(col[::-1]) for col in ssmm_membres_conv.columns.values]
ssmm_membres_conv = ssmm_membres_conv.reset_index().rename(columns={'<lambda_0>_convocation_no':'convocation_list','<lambda_1>_convocation_no':'convocation_cnt'})
ssmm_membres_conv = ssmm_membres_conv.sort_values(['convocation_cnt', 'min_convocation_no'],
              ascending = [False, True]).reset_index().drop(['index'], axis=1)
ssmm_membres_conv[:60]



,name,min_convocation_no,max_convocation_no,convocation_list,convocation_cnt
0,Денисов Илья Олегович,3,8,"[3, 4, 5, 6, 7, 8]",6
1,Краснова Анастасия Александровна,4,9,"[4, 5, 6, 7, 8, 9]",6
2,Муравьёва Анна Сергеевна,4,9,"[4, 5, 6, 7, 8, 9]",6
3,Мазитова Вероника Рустэмовна,6,11,"[6, 7, 8, 9, 10, 11]",6
4,Матвеев Дмитрий Александрович,1,5,"[1, 2, 3, 4, 5]",5
5,Остроухова Наталья Владимировна,2,6,"[2, 3, 4, 5, 6]",5
6,Савченко Анастасия Игоревна,3,7,"[3, 4, 5, 6, 7]",5
7,Малахов Николай Сергеевич,6,10,"[6, 7, 8, 9, 10]",5
8,Малиновский Георгий Андреевич,7,11,"[7, 8, 9, 10, 11]",5
9,Матвеев Алексей Викторович,1,4,"[1, 2, 3, 4]",4


In [395]:
# ssmm_membres_x_conv[ssmm_membres_x_conv['name'] == 'Матвеев Дмитрий Александрович']

In [535]:
# то же самое, но для выдвижения, а не членства

temd_df = ssmm_candidates_x_votes_info_conv_1_11
ssmm_candidates_x_conv = temd_df.sort_values(['convocation_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)

ssmm_candidates_conv = ssmm_candidates_x_conv.groupby('name').agg({'convocation_no':['min','max', lambda x: list(x), lambda x: len(list(x))]}) 

ssmm_candidates_conv.columns = ['_'.join(col[::-1]) for col in ssmm_candidates_conv.columns.values]
ssmm_candidates_conv = ssmm_candidates_conv.reset_index().rename(columns={'<lambda_0>_convocation_no':'convocation_list','<lambda_1>_convocation_no':'convocation_cnt'})
ssmm_candidates_conv = ssmm_candidates_conv.sort_values(['convocation_cnt', 'min_convocation_no'],
              ascending = [False, True]).reset_index().drop(['index'], axis=1)
ssmm_candidates_conv[:60]



,name,min_convocation_no,max_convocation_no,convocation_list,convocation_cnt
0,Денисов Илья Олегович,3,8,"[3, 4, 5, 6, 7, 8]",6
1,Краснова Анастасия Александровна,4,9,"[4, 5, 6, 7, 8, 9]",6
2,Муравьёва Анна Сергеевна,4,9,"[4, 5, 6, 7, 8, 9]",6
3,Мазитова Вероника Рустэмовна,6,11,"[6, 7, 8, 9, 10, 11]",6
4,Матвеев Дмитрий Александрович,1,5,"[1, 2, 3, 4, 5]",5
5,Остроухова Наталья Владимировна,2,6,"[2, 3, 4, 5, 6]",5
6,Савченко Анастасия Игоревна,3,7,"[3, 4, 5, 6, 7]",5
7,Малахов Николай Сергеевич,6,10,"[6, 7, 8, 9, 10]",5
8,Малиновский Георгий Андреевич,7,11,"[7, 8, 9, 10, 11]",5
9,Матвеев Алексей Викторович,1,4,"[1, 2, 3, 4]",4


In [538]:
# НА КАКИХ КУРСАХ ИЗБИРАЮТСЯ?
# - вообще
# - впервые
# - в последний раз

temp_df = ssmm_candidates_x_votes_info_conv_1_11

temp_df[['year_no','stream_no']] = pd.DataFrame(temp_df['year_stream_no'].apply(lambda x: list(split_year_stream(x))).tolist(), index=temp_df.index)

temp_df.head()


,conv_x_name,name,convocation_no,year_stream_no,academic_group_no,program_txt,announcement_link,announcement_dttm,votes_cnt,elected_flg,result_link,result_dttm,appearance_dttm,year_no,stream_no
0,1_Левин_Андрей,Левин Андрей,1,1.1,101.0,Активный участник и организатор различных меро...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,48.0,1.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1
1,1_Матвеев_Дмитрий_Александрович,Матвеев Дмитрий Александрович,1,1.1,101.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,49.0,1.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,1
2,1_Власов_Артем_Андреевич,Власов Артем Андреевич,1,1.2,107.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,36.0,1.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,2
3,1_Кириллова_Полина,Кириллова Полина,1,1.2,112.0,Мне нравится заниматься общественной деятельно...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,40.0,1.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,2
4,1_Семенов_Павел,Семенов Павел,1,1.2,112.0,"""Здравствуйте. Я так понимаю, что я уже поздно...",https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,20.0,0.0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00,1,2


In [ ]:

# - сколько раз подавался
# - на каком курсе впервые избрался (нужно тянуть номер курса)
# - на каком курсе в последний раз избрался (нужно тянуть номер курса)

# - сделать таблицу-матрицу с 11 колонками созывов и флагами членства



In [ ]:
# - по кандидату прочитать программы за все годы

## To fix ORGANIZERS

In [465]:
ssmm_organizers_info = pd.read_csv('ssmm_organizers_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_organizers_info.head()

,year_x_year_stream,organizers
0,2014_1.1,Ильёй Денисовым
1,2014_1.1,Анастасией Сурковой
2,2014_1.2,Ираклием Глунчадзе
3,2014_1.2,Анастасией Сурковой
4,2014_1.3,Ираклием Глунчадзе


In [533]:
def find_organizer_origin(organizer_name):
#     ИСПОЛЬЗУЕТСЯ ВНЕШНЯЯ ТАБЛИЦА ssmm_candidates_x_votes_info_conv_1_11
    full_name_list = list(ssmm_candidates_x_votes_info_conv_1_11['name'])
    organizer_name = organizer_name.replace('Воробьевой Наталией','Воробьёвой Натальей')
    organizer_name_list = [word.replace('Павлом','Павел').replace('Корней','Провидухина')[:-2].replace('Петр','Пётр') for word in organizer_name.split()]
#     print(organizer_name_list)
    correct_full_name = None
    for full_name in full_name_list:
        k = 0
        for word in organizer_name_list:
            if word in full_name:
                k += 1
        if k == len(organizer_name_list):
            correct_full_name = full_name
            break
    return correct_full_name

# organizer_name = 'Анастасией Сурковой'
# find_organizer_origin(organizer_name)

In [482]:
ssmm_organizers_info_names = ssmm_organizers_info.groupby(['organizers']).agg({'year_x_year_stream':'count'}).reset_index().rename(columns={'year_x_year_stream':'elections_cnt'}).sort_values(['elections_cnt'],
              ascending = [False]).reset_index().drop(['index'], axis=1)

ssmm_organizers_info_names['full_name'] = ssmm_organizers_info_names['organizers'].apply(lambda x: find_organizer_origin(x))
# ssmm_organizers_info_names[:60]
# ssmm_organizers_info_names[60:]

# баг распознавания в organizers (костыльно преодолен):
# Владиславом Бун


In [483]:
ssmm_organizers_info_upd = ssmm_organizers_info.merge(ssmm_organizers_info_names[['organizers','full_name']],left_on='organizers',right_on='organizers',how='left').rename(columns={'full_name':'organizer_name'}).drop(['organizers'],axis=1)
ssmm_organizers_info_upd = ssmm_organizers_info_upd[ssmm_organizers_info_upd['organizer_name'] != None] # не работает

ssmm_organizers_info_upd.head()
# [:60]


,year_x_year_stream,organizer_name
0,2014_1.1,Денисов Илья Олегович
1,2014_1.1,Суркова Анастасия Юрьевна
2,2014_1.2,Глунчадзе Ираклий Вахтангович
3,2014_1.2,Суркова Анастасия Юрьевна
4,2014_1.3,Глунчадзе Ираклий Вахтангович


In [481]:
ssmm_organizers_info_upd.to_csv('ssmm_organizers_info_upd.csv', sep='\t', encoding='utf-8', index=False)


## To create new tables-DICTIONARIES

In [504]:


def define_specialization(stream_no):
    if stream_no in [1,2]:
        spec = 'Фундаментальная математика'
    elif stream_no == 3:
        spec = 'Фундаментальная механика'
    elif stream_no == 4:
        spec = 'Математика и экономическая теория' # старое: Математические методы экономики
    elif stream_no == 5:
        spec = 'Фундаментальная математика и математическая физика'
    elif stream_no == 0:
        spec = 'Магистратура'
    else:
        spec = 'ERROR'
    return spec

def define_short_specialization(stream_no):
    if stream_no in [1,2]:
        spec = 'Математика'
    elif stream_no == 3:
        spec = 'Механика'
    elif stream_no == 4:
        spec = 'Эконом' # старое: Математические методы экономики
    elif stream_no == 5:
        spec = 'ФММФ'
    elif stream_no == 0:
        spec = 'Магистратура'
    else:
        spec = 'ERROR'
    return spec

# С сентября 2021 года формируется группа 241 - экспериментальная группа программы (специализации) 
# "Фундаментальная математика и математическая физика".
# О программе можно посмотреть на сайте https://fmmp.math.msu.ru/about/

# учебные планы
# https://math.msu.ru/uplans


def define_course_level(year_stream_no):
    if 'm' in year_stream_no:
        course_level = 'Старшие курсы'
    elif int(year_stream_no[0]) <= 2:
        course_level = 'Младшие курсы'
    elif int(year_stream_no[0]) >= 3:
        course_level = 'Старшие курсы'
    else:
        course_level = 'ERROR'
    return course_level

In [505]:

ssmm_dict_year_stream = ssmm_candidates_x_votes_info_conv_1_11[['year_stream_no']].drop_duplicates().sort_values(['year_stream_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)
# ssmm_dict_year_stream['kek'] = ssmm_dict_year_stream['year_stream_no'].apply(lambda x: list(split_year_stream(x)))
ssmm_dict_year_stream[['year_no','stream_no']] = pd.DataFrame(ssmm_dict_year_stream['year_stream_no'].apply(lambda x: list(split_year_stream(x))).tolist(), index=ssmm_dict_year_stream.index)
ssmm_dict_year_stream['course_level'] = ssmm_dict_year_stream['year_stream_no'].apply(lambda x: define_course_level(x))
ssmm_dict_year_stream['specialization'] = ssmm_dict_year_stream['stream_no'].apply(lambda x: define_specialization(x))
ssmm_dict_year_stream['short_specialization'] = ssmm_dict_year_stream['stream_no'].apply(lambda x: define_short_specialization(x))
ssmm_dict_year_stream

# df2[['team1','team2']] = pd.DataFrame(df2.teams.tolist(), index= df2.index)


,year_stream_no,year_no,stream_no,course_level,specialization,short_specialization
0,1.1,1,1,Младшие курсы,Фундаментальная математика,Математика
1,1.2,1,2,Младшие курсы,Фундаментальная математика,Математика
2,1.3,1,3,Младшие курсы,Фундаментальная механика,Механика
3,2.1,2,1,Младшие курсы,Фундаментальная математика,Математика
4,2.2,2,2,Младшие курсы,Фундаментальная математика,Математика
5,2.3,2,3,Младшие курсы,Фундаментальная механика,Механика
6,2.5,2,5,Младшие курсы,Фундаментальная математика и математическая фи...,ФММФ
7,3.1,3,1,Старшие курсы,Фундаментальная математика,Математика
8,3.2,3,2,Старшие курсы,Фундаментальная математика,Математика
9,3.3,3,3,Старшие курсы,Фундаментальная механика,Механика


In [506]:
ssmm_dict_year_stream.to_csv('ssmm_dict_year_stream.csv', sep='\t', encoding='utf-8', index=False)


In [528]:
def define_graduated_year(year_stream_no, year):
    if 'm' in year_stream_no:
        if year_stream_no == 'm2':
            graduated_flg = 1
        else:
            graduated_flg = 0
    elif int(year_stream_no[0]) == 5 and year < 2016:
        graduated_flg = 1
    elif int(year_stream_no[0]) == 6 and year >= 2016:
        graduated_flg = 1
    else:
        graduated_flg = 0
    return graduated_flg

def year_by_conv_no(conv_no):
    if conv_no >= 2:
        return conv_no+2010
    elif conv_no == 1:
        return 2012
    else:
        return -1


In [ ]:

# import functools
# functools.reduce(lambda x, y: )


In [531]:
ssmm_dict_years = ssmm_candidates_x_votes_info_conv_1_11[['convocation_no','year_stream_no']].drop_duplicates().sort_values(['convocation_no','year_stream_no'],
              ascending = [True,True]).reset_index().drop(['index'], axis=1)
ssmm_dict_years['year'] = ssmm_dict_years['convocation_no'].apply(lambda x: year_by_conv_no(x) )
ssmm_dict_years['graduated_flg'] = ssmm_dict_years.apply(lambda x: define_graduated_year(x['year_stream_no'], x['year']), axis=1)

ssmm_dict_graduated_years = ssmm_dict_years
ssmm_dict_graduated_years


,convocation_no,year_stream_no,year,graduated_flg
0,1,1.1,2012,0
1,1,1.2,2012,0
2,1,1.3,2012,0
3,1,2.1,2012,0
4,1,2.2,2012,0
...,...,...,...,...
194,11,5.3,2021,0
195,11,5.4,2021,0
196,11,6.1,2021,1
197,11,6.3,2021,1


In [532]:
ssmm_dict_graduated_years.to_csv('ssmm_dict_graduated_years.csv', sep='\t', encoding='utf-8', index=False)


# ПРОВЕРКИ

In [15]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

string1 = "abc"
string2 = "aef"

print(levenshtein(string1, string2))

2


In [119]:
names = list(candidates_conv_1_11_upd['name']) + list(temp_votes_conv_1_3['name']) + list(final_votes_conv_4_11['name'])

from collections import Counter

names_stat = Counter(names) #.most_common()
# names_stat

# names_uniq = list(set(names))
# names_uniq

In [120]:
# выявляем похожие имена, чтобы потом заменить РУКАМИ

for a in names_stat.keys():
    for b in names_stat.keys():
        if a != b and levenshtein(a, b) <= 4 and names_stat[a] >= names_stat[b]:
            print(levenshtein(a, b), ' - ', a, names_stat[a], ' : ', names_stat[b], b)

In [121]:
# выявляем имена без отчества, чтобы потом заменить РУКАМИ

for a in names_stat.keys():
    for b in names_stat.keys():
        if len(a) > len(b) and levenshtein(a[:len(b)], b) <= 4:
            print(a, names_stat[a], ' : ', names_stat[b], b)


Семенюк Павел Игоревич 4  :  2 Семенов Павел
Белан Андрей Владимирович 2  :  2 Левин Андрей


In [42]:
# выявляем имена с перепутанными отчествами, чтобы потом заменить РУКАМИ

for a in names_stat.keys():
    for b in names_stat.keys():
        if a != b and levenshtein(a[:12], b[:12]) <= 2 and names_stat[a] >= names_stat[b]:
            print(a, names_stat[a], ' : ', names_stat[b], b)

# Сакаев Роберт Марселевич 5  :  1 Сакаев Роберт Викторович


Коновалов Антон Александрович 2  :  1 Коновалов Дмитрий Игоревич
Семенюк Павел Игоревич 2  :  1 Семенов Павел
Порфирьева Наталия Алексеевна 1  :  1 Перфильева Нелли Андреевна
Федотов Константин Алексеевич 1  :  1 Фёдоров Константин Дмитриевич
Алиев Рамиль Камиль Оглы 1  :  1 Алеев Дамиль Павлович
Перфильева Нелли Андреевна 1  :  1 Порфирьева Наталия Алексеевна
Фёдоров Константин Дмитриевич 1  :  1 Федотов Константин Алексеевич
Третьяков Пётр Максимович 3  :  2 Третьяков Артём Алексеевич
Саметов Максим Алексеевич 2  :  1 Савелов Максим Павлович
Лобанова Алёна Валерьевна 3  :  1 Лобанова Анастасия Алексеевна
Алеев Дамиль Павлович 1  :  1 Алиев Рамиль Камиль Оглы
